---
jupyter:
  jupytext:
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3
    language: python
    name: python3
---

<!-- #region id="bd6d5a06" -->
# Table des matières
1. [Lecture et préparation des données](#lecture-et-préparation-des-données)
1. [Entraînement d'un classificateur de type forêt aléatoire](#entraînement-dun-classificateur-de-type-forêt-aléatoire)
1. [Affichage des statistiques de classification](#affichage-des-statistiques-de-classification)
1. [Identification des variables les plus importantes pour diagnostiquer le diabète](#identification-des-variables-les-plus-importantes-pour-diagnostiquer-le-diabète)

# Attention!
Ne lancez pas l'exécution automatique du notebook en entier en cliquant sur le bouton **Tout exécuter**. L'exécution serait interrompue, car certaines cellules exigent une entrée de votre part!

Il faut simplement exécuter le notebook, une cellule à la fois, et entrer quelques lignes de code lorsque demandées. Il est inutile de sauter ces cellules pour aller aux suivantes car celles-ci ont justement besoin de votre input!

Importons d'abord les librairies nécessaires.
<!-- #endregion -->



In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sns.set(color_codes=True)


seed = 42
np.random.seed(seed)



<!-- #region id="f83e124a" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/diabetes-blood-sample.jpeg"  width="500" />
    <div>
    <font size="0.5">Image Source: https://medicalfitness.com.au/type-1-and-2-diabetes-australia/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="4a5eb103" -->
Dans ce module, nous allons appliquer la classification par forêt aléatoire afin de prédire si un sujet
est diabétique ou non en fonction de variables mesurés à la suite d'une visite médicale typique (prise
de pression, de sang, d'urine, de poids, etc.)

Ce type d'analyse a mené à une meilleure compréhension du diabète. C'est un bel exemple de l'utilisation de
l'apprentissage automatique en médecine. Dans ce problème, nous sommes confrontés à des données
disparates (taux de glucose dans le sang, indice de masse corporelle, épaisseur de la peau, etc.)

Comme on l'a vu dans les précédentes sections, la classification sert à prédire une *réponse* $y$, qui est
une variable ordinale ou catégorielle (c.-à-d. une classe), en fonction de plusieurs variables $x_{i}$

$$y=f(x_{1}, \cdots, x_{N}, \Theta)$$

où $\Theta$ représente l'ensemble des paramètres de la fonction $f$.

Il est généralement impossible de modéliser exactement cette fonction dû au grand nombre de phénomènes
impliqués lors de la prise de données. La modélisation par forêt aléatoire permet de passer outre à la
modélisation analytique. Elle permet de modéliser un phénomène impliquant un très grand nombre d'interactions entre les
différentes variables sans avoir à les spécifier explicitement dans un modèle mathématique. C'est une des raisons
qui expliquent leur immense champ d'applications en sciences de la vie et en sciences sociales par exemple.

<!-- #endregion -->

<!-- #region id="8aadf220" -->
# <a id=lecture-et-préparation-des-données>Lecture et préparation des données</a>
<!-- #endregion -->

<!-- #region id="b856da26" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/classifier-model.png"  width="500" />
    <div>
    <font size="0.5">Image Source: Google Image</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="7897e110" -->
Le  jeu de données contient neuf variables, continues ou ordinales, qui ont été mesurées chez 768 sujets:


- Pregnancies: nombre de grossesses,
- Glucose: taux de glucose,
- BloodPressure: pression artérielle,
- SkinThickness: épaisseur de la peau,
- Insulin: taux d'insuline,
- BMI: indice de masse corporelle (IMC), 
- DiabetesPedigreeFunction: facteur de diabète,
- Age: âge,
- Outcome: résultat.


La dernière, Outcome, contient la réponse binaire que l'on veut prédire. Le sujet est atteint
du diabète lorsque Outcome=1.

Les données originales ont été extraites du jeu de données
[diabetes](https://www.kaggle.com/uciml/pima-indians-diabetes-database). Celle-ci contient toutefois
plusieurs valeurs manquantes. Elle a été nettoyée en remplaçant, pour chaque variable sauf Pregnancies et Outcome, les valeurs manquantes par la valeur médiane des valeurs présentes. Dans ce qui suit, nous allons utiliser les données nettoyées.

Le but de notre analyse est de déterminer si un sujet est diabétique, ou non, en fonction de ces variables.

<!-- #endregion -->



In [ ]:
# Lecture du jeu de données en format CSV

df = pd.read_csv("/pax/shared/GIF-U014/diabetes_nettoyée.csv")


In [ ]:
# Affichage de statistiques sur les données

df.describe(include="all")



<!-- #region id="a0ffd894" -->
On voit que les valeurs couvrent différents ordres de grandeur. Il faudra les normaliser pour cette raison. Bien que les arbres décisionnels et les forêts aléatoires ne soient pas affectés par la normalisation, c'est une bonne habitude de normaliser ses données.
<!-- #endregion -->

<!-- #region id="0739ca01" -->
La colonne `'Unnamed: 0'` contient un indice allant de 0 à 767. Elle est inutile.
<!-- #endregion -->



In [ ]:
df.pop("Unnamed: 0")


In [ ]:
# Séparation des variables X et de la réponse y du jeu de données

X = df.drop(["Outcome"], axis=1)
y = df.Outcome

# Liste des variables utilisées
feature_list = list(X.columns)



<!-- #region id="89346c43" -->
Affichons le nombre de données pour chacune pour vérifier s'il y a un débalancement.
<!-- #endregion -->



In [ ]:
y.value_counts().plot(kind="bar").set_title("Diabetes Outcome")



<!-- #region id="55662d08" -->
La classe 0 comprend environ $65~\%$ des données. Il y a un léger débalancement des données dont nous allons tenir compte pour la suite de l'analyse.
<!-- #endregion -->

<!-- #region id="ac0c6c85" -->
Les données d'entraînement vont servir à entraîner le classificateur. C'est-à-dire, à estimer la valeur du seuil  $\tau$  de chaque noeud, dans chaque arbre décisionnel de la forêt. Les données de test vont ensuite servir à mesurer les performances du modèle entraîné à prédire correctement si une personne est atteinte ou non du diabète.

L'échantillonnage stratifié fait en sorte que les deux ensembles de données contiennent les mêmes proportions de sujets diabétiques et sains.
<!-- #endregion -->



In [ ]:
# Génération stratifiée des ensembles d'entraînement et de test

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=seed
)



<!-- #region id="9815a089" -->
Par la suite, nous allons normaliser les données.

Les paramètres de la fonction de normalisation doivent être calculés à partir des données
d'entraînement uniquement. La même fonction est ensuite appliquée aux ensembles d'entraînement et de test. Il ne
faut pas normaliser les données avant de générer les ensembles d'entraînement et de test.

Selon l'approche du `StandardScaler`, chaque colonne $x_i$ d'une matrice $X$ est transformée de la façon suivante

$$x'_i = \dfrac{x_i-\mu_i}{\sigma_i}$$

où $\mu_i$ et $\sigma_i$ sont la moyenne et l'écart-type des valeurs de $x_i$, calculés avec les données d'entraînement.
<!-- #endregion -->



In [ ]:
scaler = StandardScaler()

X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)



<!-- #region id="107d9537" -->
Pourquoi les paramètres de la fonction de normalisation doivent-ils être calculés uniquement à partir des données
d'entraînement?
<!-- #endregion -->

<!-- #region id="8260384f" -->
# <a id=entraînement-dun-classificateur-de-type-forêt-aléatoire>Entraînement d'un classificateur de type forêt aléatoire</a>
<!-- #endregion -->

<!-- #region id="dd63ebb5" -->
Initialiser une forêt aléatoire de classification ([`RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)) (`# À remplir`) avec les paramètres suivants:

- une forêt aléatoire de `300` estimateurs (*estimators*),
- une profondeur des arbres maximales de `4`, et
- un nombre minimal d'exemples (*samples*) de `4` pour diviser une feuille (*internal node*).
<!-- #endregion -->



In [ ]:
clf = # À remplir


In [ ]:
# Entraînement du classificateur avec les données d'entraînement

clf.fit(X_train_s, y_train)



<!-- #region id="2c5266f0" -->
# <a id=affichage-des-statistiques-de-classification>Affichage des statistiques de classification</a>
<!-- #endregion -->

<!-- #region id="e13de826" -->
Nous allons maintenant calculer quelques métriques pour évaluer les performances en classification. Celles-ci sont
décrites dans le module sur les métriques de qualité en classification. Il est toutefois utile de les décrire ici brièvement afin de mieux comprendre nos résultats de classification:


- Exactitude (*accuracy*): C'est la fraction des prédictions (sujets avec ou sans diabète) qui se sont avérées exactes.
- Précision (*precision*): C'est la fraction des prédictions positives (sujets avec diabète) qui se sont avérées exactes.
- Rappel (*recall*): C'est la fraction des sujets positifs (avec diabète) qui ont été détectés.
- Score $F_1$: C'est la moyenne harmonique de la précision et du rappel. Il est défini comme suit.


$$F_1=\frac{2 *\text{Précision} *\text{Rappel}}{\text{Précision} + \text{Rappel}}$$

Dans une situation où il est difficile de choisir entre les métriques $\text{Précision}$ et $\text{Rappel}$, le
score $F_1$, qui les combine, est souvent préféré. Pourquoi ne pas simplement utiliser la moyenne arithmétique des deux métriques? Cela est dû au fait que chacune est un pourcentage et donc un ratio. On doit utiliser la moyenne harmonique
pour calculer la moyenne de ratios.

Voici un exemple. Supposons que la distance entre deux villes est de 120 km. À l'aller, le trajet dure 3 h et au retour il dure 2 h. La vitesse est de $v_a=40$ km/h à l'aller, et de $v_r=60$ km/h au retour. La vitesse moyenne est-elle réellement de $\bar{v}=(v_a+v_b)/2=50$ km/h? Non! Une distance totale de 240 km a été parcourue en 5 h. La vitesse moyenne est de
$\bar{v} = 240 \text{ km}/5\text{ h}=48\text{ km/h}$. La bonne réponse est la moyenne harmonique:


$$\begin{align}
\bar{v} &= \frac{2 v_{a} v_{r}}{v_{a} +v_{r}} \\
        &= \frac{2 \times 40 \times 60 }{40 + 60} \\
        &= 48
\end{align}$$

La différence entre 48 km/h et 50 km/h n'est pas grande, mais elle est réelle.
> À noter que l'on calcule souvent la moyenne arithmétique de pourcentages, qui sont des ratios. Ce n'est pas la bonne façon de procéder, mais c'est dans les habitudes.

Comme vous le voyez, il y a des subtilités dans les définitions des différentes métriques, mais elles sont importantes.
<!-- #endregion -->



In [ ]:
# Prédictions pour les ensembles d'entraînement et de test

pred_train = clf.predict(X_train_s)
pred_test = clf.predict(X_test_s)


In [ ]:
# Comparaison des valeurs d'exactitude

print(
    "Exactitude sur les données d'entraînement: %0.1f %%"
    % (100 * accuracy_score(y_train, pred_train))
)

print(
    "Exactitude sur les données de test: %0.1f %%\n"
    % (100 * accuracy_score(y_test, pred_test))
)



<!-- #region id="1dc1bc77" -->
L'exactitude en entraînement est d'environ $83~\%$ alors que celle en test est d'environ $72~\%$.
Ainsi $72~\%$ des prédictions effectuées sur de nouveaux sujets, avec ou sans diabète, sont correctes.
<!-- #endregion -->

<!-- #region id="6653ad08" -->
Maintenant, affichons un rapport sur les statistiques de classification. Celui-ci permet d'obtenir une comparaison plus fine des performances en test.
<!-- #endregion -->



In [ ]:
print(classification_report(y_test, pred_test))



<!-- #region id="ef1c3780" -->
La proportion des résultats positifs qui correspondent réellement à des sujets diabétiques (la précision) est de $64~\%$.
La proportion des sujets diabétiques détectés et qui le sont réellement (le rappel ou *recall*) est de $46~\%$.

On observe des F1-scores de $80~\%$ pour les cas normaux et de $54~\%$ pour les cas diabétiques.
<!-- #endregion -->

<!-- #region id="ec376d01" -->
# <a id=identification-des-variables-les-plus-importantes-pour-diagnostiquer-le-diabète>Identification des variables les plus importantes pour diagnostiquer le diabète</a>
<!-- #endregion -->

<!-- #region id="0188acbb" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/inspector-with-magnifying-glass.jpeg"  width="200" />
    <div>
    <font size="0.5">Image Source: http://clipart-library.com/clipart/1416328.htm</font>
    </div>
</div>

<!-- #endregion -->

<!-- #region id="eb1eb36d" -->
> À noter que cette section est similaire à celle dans le module montrant un exemple de régression par forêt aléatoire. Toutefois,
nous allons la revisiter, car les concepts qui y sont expliqués sont très utiles dans la pratique.

Les variables $x_{i}$ les plus importantes sont les plus utilisées pour prendre des décisions à travers les arbres décisionnels constituant la forêt.

L'importance de chaque variable correspond au nombre de fois qu'elle est utilisée dans la forêt aléatoire pour prendre une décision. Elle prend une valeur entre 0 et 1, où 0 indique qu'elle n'est jamais utilisée, et où 1 indique qu'elle est la seule utilisée parmi toutes. Dans ce dernier cas, la variable permet de prédire parfaitement la réponse.
La somme des importances sur l'ensemble des variables vaut 1.
<!-- #endregion -->



In [ ]:
# Calcul de l'importance de chaque variable

importances = list(clf.feature_importances_)

# Le nom de chaque variable est associé à son importance
feature_importances = [
    (feature, round(importance, 2))
    for feature, importance in zip(feature_list, importances)
]

# Ordonnancement des valeurs d'importance en ordre décroissant
feature_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)



<!-- #region id="3a4a6006" -->
On veut déterminer lesquelles sont les plus importantes et combien il y en a. Pour cela, nous allons afficher un diagramme d'importance des variables du jeu de données.
<!-- #endregion -->



In [ ]:
indices = np.argsort(importances)[::-1]

plt.style.use("fivethirtyeight")

var = list(range(len(importances)))
plt.bar(var, np.array(importances)[indices.astype(int)], orientation="vertical")
plt.xticks(var, np.array(feature_list)[indices.astype(int)], rotation="vertical")
plt.ylabel("Importance", fontsize=16)
plt.xlabel("Variable", fontsize=16)
plt.title("Importance des variables", fontsize=16)



<!-- #region id="b934ad71" -->
On remarque qu'il n'y a pas de séparation franche entre les variables. Toutefois, il semble y avoir un léger « coude » dans la distribution à la quatrième variable. Ainsi, les quatre premières variables semblent être les plus
importantes, soit en ordre décroissant:

- le taux de glucose,
- l'indice de masse corporelle (BMI),
- l'âge,
- le taux d'insuline.

Notez bien que ces résultats ne sont valides que pour ce  jeu de données et pour le type de prétraitement utilisé. En effet, l'ordre précis des variables indicatrices peut légèrement changer en fonction du type de prétraitement ou du classificateur utilisés.
<!-- #endregion -->
